In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html

import pandas as pd
import numpy as np
import plotly.graph_objs as go

In [2]:

url = 'https://data.cityofnewyork.us/resource/nwxe-4ae8.json?$limit=10000'
trees = pd.read_json(url)
#trees.dropna(inplace = True)
trees.head(10)

,address,bbl,bin,block_id,boro_ct,borocode,boroname,brch_light,brch_other,brch_shoe,...,tree_dbh,tree_id,trnk_light,trnk_other,trunk_wire,user_type,x_sp,y_sp,zip_city,zipcode
0,108-005 70 AVENUE,4.022210e+09,4052307.0,348711,4073900,4,Queens,No,No,No,...,3,180683,No,No,No,TreesCount Staff,1.027431e+06,202756.7687,Forest Hills,11375
1,147-074 7 AVENUE,4.044750e+09,4101931.0,315986,4097300,4,Queens,No,No,No,...,21,200540,No,No,No,TreesCount Staff,1.034456e+06,228644.8374,Whitestone,11357
2,390 MORGAN AVENUE,3.028870e+09,3338310.0,218365,3044900,3,Brooklyn,No,No,No,...,3,204026,No,No,No,Volunteer,1.001823e+06,200716.8913,Brooklyn,11211
3,1027 GRAND STREET,3.029250e+09,3338342.0,217969,3044900,3,Brooklyn,No,No,No,...,10,204337,No,No,No,Volunteer,1.002420e+06,199244.2531,Brooklyn,11211
4,603 6 STREET,3.010850e+09,3025654.0,223043,3016500,3,Brooklyn,No,No,No,...,21,189565,No,No,No,Volunteer,9.909138e+05,182202.4260,Brooklyn,11215
5,8 COLUMBUS AVENUE,1.011310e+09,1076229.0,106099,1014500,1,Manhattan,No,No,No,...,11,190422,No,No,No,Volunteer,9.884187e+05,219825.5227,New York,10023
6,120 WEST 60 STREET,1.011310e+09,1076229.0,106099,1014500,1,Manhattan,No,No,No,...,11,190426,No,No,No,Volunteer,9.883112e+05,219885.2785,New York,10023
7,311 WEST 50 STREET,1.010410e+09,1086093.0,103940,1012700,1,Manhattan,No,No,No,...,9,208649,No,No,No,Volunteer,9.877691e+05,217157.8561,New York,10019
8,65 JEROME AVENUE,NaN,NaN,407443,5006400,5,Staten Island,No,No,No,...,6,209610,No,No,No,TreesCount Staff,9.630732e+05,156635.5542,Staten Island,10305
9,638 AVENUE Z,3.072350e+09,3320727.0,207508,3037402,3,Brooklyn,No,No,No,...,21,192755,No,No,No,TreesCount Staff,9.926537e+05,152903.6306,Brooklyn,11223


In [3]:
trees.shape


(10000, 45)

In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

tree_app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

tr1 = trees[['boroname', 'spc_common', 'health']]
tr2 = trees[['spc_common', 'health', 'steward']]
hs = tr2.groupby(['spc_common', 'health'])

species = tr1['spc_common'].unique()
borough = tr1['boroname'].unique()
health = tr2['health'].unique()
colors = ["#006D2C", "#31A354", "#74C476", "#90D691"]
tg = tr1.groupby(['spc_common', 'boroname'])
#my_graph = tg.get_group(('tulip-poplar', 'Brooklyn'))['health'].value_counts().plot.pie(x = 'health', colors=colors, autopct='%1.1f%%', shadow=True, startangle=140)

tree_app.layout = html.Div(children=[
    html.H1(children='Module 4: Tree Health App'),
    html.H5(children='''In this module we’ll be looking at data from the New York City tree census:  
 
                        https://data.cityofnewyork.us/Environment/2015-Street-Tree-Census-Tree-Data/uvpi-gqnh
 
                        This data is collected by volunteers across the city, and is meant to catalog information about every single tree in the city.
                        The idea is to build a dash app for a arborist studying the health of various tree species (as defined by the variable ‘spc_common’) 
                        across each borough (defined by the variable ‘borough’). This arborist would like to answer the following two questions for each species and in each borough: 
                        '''),

    html.Div(children='''
        Question 1:  What proportion of trees are in good, fair, or poor health according to the ‘health’ variable?
    '''),

    dcc.Dropdown(
         id="tree_type",
         options=[{'label': i, 'value': i} for i in species],
         placeholder = 'Select A Tree'
         ),
    
    dcc.Dropdown(
         id="borough",
         options=[{'label': i, 'value': i} for i in borough],
         placeholder = 'Select A Borough'
         ),
       
     dcc.Graph(
        id = 'health_graph'    
    ),
     
    html.Div(children='''
        Question 2:   Are stewards (steward activity measured by the ‘steward’ variable) having an impact on the health of trees? 
     '''),
     
     dcc.Dropdown(
         id="tree_type2",
         options=[{'label': i, 'value': i} for i in species],
         placeholder = 'Select A Tree'
         ),
    
    dcc.Dropdown(
         id="health",
         options=[{'label': i, 'value': i} for i in health],
         placeholder = 'Select A Health Element'
         ),
    
        dcc.Graph(
        id = 'steward_graph'    
    )
    
])

@tree_app.callback(
    dash.dependencies.Output('health_graph', 'figure'),
    [dash.dependencies.Input('tree_type', 'value'),
    dash.dependencies.Input('borough', 'value')])

def update_graph(selected, selected2):
    trace = go.Pie(labels = tg.get_group((selected, selected2))['health'].unique() , values = tg.get_group((selected, selected2))['health'].value_counts())
    data = [trace]
    fig = go.Figure(data = data)
    return fig


@tree_app.callback(
    dash.dependencies.Output('steward_graph', 'figure'),
    [dash.dependencies.Input('tree_type2', 'value'),
    dash.dependencies.Input('health', 'value')])

def update_graph2(selected, selected2):
   trace2 = go.Bar(x = hs.get_group((selected, selected2))['steward'].unique() , y = hs.get_group((selected, selected2))['steward'].value_counts(), marker={'color': colors})
   data = [trace2]
   fig2 = go.Figure(data = data)
   return fig2


if __name__ == '__main__':
    tree_app.run_server()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [03/May/2020 19:36:51] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2020 19:36:51] "GET /_dash-component-suites/dash_renderer/polyfill@7.v1_3_0m1588548947.8.7.min.js HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2020 19:36:51] "GET /_dash-component-suites/dash_core_components/dash_core_components.v1_9_0m1588548948.min.js HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2020 19:36:51] "GET /_dash-component-suites/dash_renderer/react-dom@16.v1_3_0m1588548947.13.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2020 19:36:51] "GET /_dash-component-suites/dash_renderer/prop-types@15.v1_3_0m1588548947.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2020 19:36:51] "GET /_dash-component-suites/dash_renderer/react@16.v1_3_0m1588548947.13.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2020 19:36:51] "GET /_dash-component-suites/dash_html_components/dash_html_components.v1_0_3m1588548949.min.js HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2020 19:36:

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/zhouw/anaconda2/lib/python2.7/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/zhouw/anaconda2/lib/python2.7/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/zhouw/anaconda2/lib/python2.7/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/zhouw/anaconda2/lib/python2.7/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/zhouw/anaconda2/lib/python2.7/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/Users/zhouw/anaconda2/lib/python2.7/site-packages/dash/dash.py", line 967, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "/Users/

127.0.0.1 - - [03/May/2020 19:36:53] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/zhouw/anaconda2/lib/python2.7/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/zhouw/anaconda2/lib/python2.7/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/zhouw/anaconda2/lib/python2.7/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/zhouw/anaconda2/lib/python2.7/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/zhouw/anaconda2/lib/python2.7/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/Users/zhouw/anaconda2/lib/python2.7/site-packages/dash/dash.py", line 967, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "/Users/

127.0.0.1 - - [03/May/2020 19:36:53] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [03/May/2020 19:36:53] "GET /_dash-component-suites/dash_core_components/async-dropdown.v1_9_1m1586525729.js HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2020 19:36:53] "GET /_dash-component-suites/dash_core_components/async-plotlyjs.v1_9_1m1586525729.js HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2020 19:36:53] "GET /_dash-component-suites/dash_core_components/async-graph.v1_9_1m1586525729.js HTTP/1.1" 200 -
